<a href="https://colab.research.google.com/github/chibuezedev/Machine-learning/blob/main/cloth-classification-using-pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    train=True,
    transform=ToTensor(),
    download=True,
    root="data"
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    train=False,
    transform=ToTensor(),
    download=True,
    root="data"
)

100%|██████████| 26421880/26421880 [00:07<00:00, 3307878.34it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 203451.73it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3769558.24it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 3455317.17it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
batch_size = 64

# Create data loaders.
training_loader = DataLoader(training_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=64)

for X, y in test_loader:
  print(f"Shape of X [N C H W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break




Shape of X [N C H W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


To define a neural network in PyTorch, we create a class that inherits from nn.Module. We define the layers of the network in the __init__ function and specify how data will pass through the network in the forward function. To accelerate operations in the neural network, we move it to the GPU or MPS if available.

In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.Flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.Flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (Flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Optimizing the Model Parameters
To train a model, we need a loss function and an optimizer.

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

In [14]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  print(f"size: {size}")
  for batch, (X, y) in enumerate(dataloader):
    X = X.to(device)
    y = y.to(device)

    #compute prediction error
    prediction = model(X)
    loss = loss_fn(prediction, y)

    #backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


check the model’s performance against the test dataset to ensure it is learning.

In [15]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            prediction = model(X)
            test_loss += loss_fn(prediction, y).item()
            correct += (prediction.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (epochs). During each epoch, the model learns parameters to make better predictions. We print the model’s accuracy and loss at each epoch; we’d like to see the accuracy increase and the loss decrease with every epoch.

In [16]:
epoch = 5
for t in range(epoch):
  print(f"Epoch {t+1}\n-------------------------------")
  train(training_loader, model, loss_fn, optimizer)
  test(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
size: 60000
loss: 2.301722 [    0/60000]
loss: 2.196219 [ 6400/60000]
loss: 1.919408 [12800/60000]
loss: 1.602865 [19200/60000]
loss: 1.206242 [25600/60000]
loss: 1.089895 [32000/60000]
loss: 1.010409 [38400/60000]
loss: 0.888665 [44800/60000]
loss: 0.871484 [51200/60000]
loss: 0.801176 [57600/60000]
Test Error: 
 Accuracy: 70.4%, Avg loss: 0.797208 

Epoch 2
-------------------------------
size: 60000
loss: 0.793278 [    0/60000]
loss: 0.839569 [ 6400/60000]
loss: 0.590993 [12800/60000]
loss: 0.796638 [19200/60000]
loss: 0.679180 [25600/60000]
loss: 0.645485 [32000/60000]
loss: 0.705061 [38400/60000]
loss: 0.694501 [44800/60000]
loss: 0.680406 [51200/60000]
loss: 0.634787 [57600/60000]
Test Error: 
 Accuracy: 78.1%, Avg loss: 0.632338 

Epoch 3
-------------------------------
size: 60000
loss: 0.558544 [    0/60000]
loss: 0.660779 [ 6400/60000]
loss: 0.436486 [12800/60000]
loss: 0.679563 [19200/60000]
loss: 0.600478 [25600/60000]
loss: 0.567539 

A common way to save a model is to serialize the internal state dictionary (containing the model parameters).

In [17]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


The process for loading a model includes re-creating the model structure and loading the state dictionary into it.

In [18]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

Model Usage for prediction

In [25]:

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]



# Set model to evaluation mode
model.eval()


sample_index = 1  # Change this index to select a different sample
x, y = test_data[sample_index][0], test_data[sample_index][1]

# Make prediction without gradient calculation
with torch.no_grad():
    x = x.to(device)
    prediction = model(x.unsqueeze(0))  # Add batch dimension

    # Get predicted and actual classes
    predicted, actual = classes[prediction.argmax(dim=1).item()], classes[y]

    print(f'Predicted: "{predicted}", Actual: "{actual}"')


Predicted: "Pullover", Actual: "Pullover"
